In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#input story here
doc =""""""


In [ ]:
input entities here
Entities = """"""

In [4]:
for e in Entities:
    if Entities[e].lower()== 'player':
        player_name = e.lower()
        break;
storyname = ''

In [ ]:
##segmentation module
story = doc.lower()
segments = []
locations = []
for entity in Entities:
    if Entities[entity] == 'location':
        locations.append(entity.lower())

locations_no = {}

# Get number of every location
for location in locations:
    loc_no = story.find(location)
    if loc_no is not  -1:
        locations_no[location] = loc_no



# sort location by number
sorted_locations = []
sorted_loc = {k: v for k, v in sorted(locations_no.items(), key=lambda item: item[1])}

for loc in sorted_loc:
    sorted_locations.append(loc)
# print(sorted_locations)

# Get first segment #
loc1 = story.find(location) + len(location)+1
loc2 = story.find(sorted_locations[1])


last_stop  = story[:loc2][::-1].find(".")   # last stop before location2
first_boundary = loc2 - last_stop+1

story_after_loc1 = story[first_boundary:]

segment_1 = story[:first_boundary]
segments.append(segment_1)
# print("segment"+"0", ":", segment_1)

monitor = 1
seg_length = len(segment_1)

#Get for segment two then replicate for others
if monitor > 0:

    #last and next location
    loc1 = story_after_loc1.find(sorted_locations[1])+len(sorted_locations[1])+1
    loc2 = story_after_loc1.find(sorted_locations[2])


    # inverse
    rev_story = story_after_loc1[:loc2][::-1]
    nxt_stop = rev_story.find(".")
  
    
    # reverse inverse
    segment = rev_story[nxt_stop:][::-1]
    seg_length = seg_length + len(segment)
    segments.append(segment)
#     print("segment"+"1"+ ":"+ segment , "\n")


    story_after_loc1 = story[seg_length:]
    # print(story_after_loc1)

#Get segments from three to four
    #to check if a location has gone with the initial location's sentence
    #check if which of the locations is present in the sentence and remove from the list
locs_length = len(sorted_locations[2:])
for a, location in enumerate(sorted_locations[2:], 2):
            #if location is not the last location

    if a < locs_length+1:
                # this and next location
        loc1 = story_after_loc1.find(sorted_locations[a]) + len(sorted_locations[a]) + 1
        loc2 = story_after_loc1.find(sorted_locations[a + 1])


        # invert
        rev_story = story_after_loc1[:loc2][::-1]
        nxt_stop = rev_story.find(".")

        # reverse the inverse
        segment = rev_story[nxt_stop:][::-1]
        seg_length = seg_length + len(segment)
#         print("segment" + str(a)+ ":"+ segment, "\n")
        segments.append(segment)

        story_after_loc1 = story[seg_length:]

    else:
        segments.append(story_after_loc1)
#         print("segment" + str(a)+ ":"+ story_after_loc1, "\n")




segments = [segment for segment in segments if len(segment)>1]
for i, segment in enumerate(segments):
    print(str(i)+":"+segment)

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

#load segments
all_segments = [nlp(segment) for segment in segments ]

In [ ]:
# random cost
import random
def random_cost():
    return random.randint(1,4)
print(random_cost())

In [ ]:
##helpers
# lemmatizes
def lemmat(action):
    action  = nlp(action)
    for token in action:
        return token.lemma_
        break;
space = ' '

#turns entities into predicate labels
def label(entity):
    return "?"+str(entity)[:3]


#remove duplicate from list
def unduplicate(arr):
    arr  = list(dict.fromkeys(arr))
    return arr

#get past
verbs_data = pd.read_csv("../input/english-verbs-and-their-tenses/verb.csv")
verbs= verbs_data.copy()
verbs = verbs[['convolute', 'convoluted']]
def get_past(action):
    act= lemmat(action)
    verb  = verbs.loc[verbs['convolute'] == act]
    
    try:
        return verb.convoluted.values[0]
    except:
        return action

In [ ]:
#function to check if word is among the entities
def isEntity(word):
    re = False
    for entity in Entities:
        if entity is str(word):
            re  = True
            break; 
    return re

#returns entity label
def ent_value(ent):
    ent = str(ent).lower()
    value = None
    for entity in Entities:
        if entity.lower() == ent:
            value = Entities[entity]
            break
    return value

In [ ]:
#import and clean story
import spacy
from spacy.symbols import nsubj, VERB, NOUN
from spacy import displacy
nlp = spacy.load("en_core_web_lg")
story = nlp(doc.lower())

In [ ]:
#returns effects
def dob_effects(subsegment):
    doc = nlp(subsegment[1])
    doc_param = subsegment[2]
    doc_action = subsegment[0]
    for token in doc:
        if token.dep_ is "nsubj"and token.text == player_name and token.head.pos is VERB :
            action = token.head
            child = [child for child in action.children if child.dep_ == 'dobj']
            if child:
                return (get_past(str(action))+ space+ label(ent_value(child[0].lower())))
            else:
                return (get_past(str(doc_action)))

In [ ]:
# save the segment number and the pobjs
#returns effect of subseg and count of effects in the sub_seg
def get_dob_effect(subsegment):
#     doc = doc.lower()
    doc = nlp(subsegment[1])

    pobjs = []
    act_pobjs = []
    for token in doc:
        if token.dep_ is 'pobj' or token.dep_ is 'dobj' or token.dep_ is 'nsubj':
            pobjs.append([token, token.i])
        
    for token in doc:
        if token.dep_ is "nsubj"and token.text == player_name and token.head.pos is VERB :
            action = token.head
            index = action.i
            get_pobj = [[action, pobj[0]] for pobj in pobjs if index<pobj[1]]
            
            if get_pobj:
                act_pobjs.append(get_pobj[0])
            
    if act_pobjs:
        for act in act_pobjs:
            if str(subsegment[0]) == str(act[0]):
                eff = act 
                print(eff)

        effect = get_past(str(eff[0]))+ space+ label(ent_value(eff[1]))
        return(effect)
    return ("none")
#    print(effect)
    
# get_dob_effect(subsegments[1])

In [ ]:
def get_obj(subsegment):
#     doc = doc.lower()
    doc = nlp(subsegment[1])

    pobjs = []
    act_pobjs = []
    for token in doc:
        if token.dep_ is 'pobj' or token.dep_ is 'dobj' or token.dep_ is 'nsubj':
            pobjs.append([token, token.i])
        
    for token in doc:
        if token.dep_ is "nsubj"and token.text == player_name and token.head.pos is VERB :
            action = token.head
            index = action.i
            get_pobj = [[action, pobj[0]] for pobj in pobjs if index<pobj[1]]
            
            if get_pobj:
                act_pobjs.append(get_pobj[0])
            
    if act_pobjs:
        for act in act_pobjs:
            if str(subsegment[0]) == str(act[0]):
                eff = act 
            

        objct = eff[1]
        return(objct)
    return ("none")
#    print(effect)
    
# get_dob_effect(subsegments[1])

In [ ]:
# #explicit effect
# Explicit = ['induce', 'give rise', 'stir up', 'create', 'start', 'launch', 'produce', 'generate', 'effect', 'arouse', 'provoke', 'elicit', 'bring about','lead',
# 'lead to', 'trigger', 'derive', 'associate', 'relate', 'link', 'stem from', 'originate', 'bring forth', 'lead up', 'trigger off', 'bring on', 'result from',
#             'set up', 'commence', "set off", 'set in motion', 'bring on', 'conduce to', 'educe', 'originate in', 'lead off', 'spark', 'spark off', 'evoke', "link up", "activate", 'actuate'
#            "put forward", "give birth to", "fire up", "unlease"]

# def get_ex_effect(subs,seg):
#     for verb in Explicit:
#         a = seg.find(verb)
#         if(a>=0):
#             return (get_past(lemmat(verb)))
#         else:
#             return None

In [ ]:
# #implicit Effects
# from nltk.corpus import wordnet as wn
# def imp_effects(subsegment):
#     action = subsegment[0]
#     for synset in wn.synsets(action):
#         a = synset.definition()
#         if(a.find('cause to')>=0):
#             return nlp(a)[2]
        
# imp_effects(["kill"])

In [ ]:
#subsegmentation module
#subsegmentation module

all_subsegments = []
all_actions = {}
all_actionss = [] 
lemma_actionss = []
objects = []
for count, segment in enumerate(segments):
    segment = nlp(segment)
    actions = []
    
    action = ''
    
    for token in segment:
        if token.dep_ is "nsubj"and token.text == player_name and token.head.pos is VERB:
            act = [token.head.text, token.i]
            actions.append(act)

            all_actions[count] = actions 
            all_actionss.append([count, token.head.text])
            
            action = token.head.text
            act_obj = get_obj([action,str(segment)])
            objects.append(act_obj)
#             print(act_obj)
        
for i, actions in enumerate(all_actionss):
    action = actions[1] #current action
    act_segment = str(all_segments[actions[0]]) #segment containing the action
    if(i<len(all_actionss)-1):
        nxtaction = all_actionss[i+1] #next action
        #find location of the current action
        act_loc = act_segment.find(action)
        #reverse the segment
        begtoact = act_segment[:act_loc]
        #find where the player is mentioned last between the beginnig and the action
        r_player = player_name[::-1]
        player_loc = len(begtoact)-begtoact[::-1].find(r_player)-len(player_name) 
#       the last name where the player is mentioned before the action
        if(act_segment[act_loc:].find('.')>-1):
            nxt_stop = len(act_segment[:act_loc]) + act_segment[act_loc:].find('.')+1
            print(i,act_segment[player_loc:nxt_stop], "\n")
            all_subsegments.append(act_segment[player_loc:nxt_stop])
        else:
            print(i, act_segment[player_loc:], "\n")
            all_subsegments.append(act_segment[player_loc:])

            
        
    else: 
        act_loc = act_segment.find(action)
        
        begtoact = act_segment[:act_loc]
        #find where the player is mentioned last between the beginnig and the action
        r_player = player_name[::-1]
        player_loc = len(begtoact)-begtoact[::-1].find(r_player)-len(player_name) 
#         print(act_segment[player_loc:])
#       the last name where the player is mentioned before the action
#         print(len(act_segment),len(act_segment[act_loc:]) )
        if(act_segment[act_loc:].find('.')>-1):
            nxt_stop = len(act_segment[:act_loc]) + act_segment[act_loc:].find('.')+1
            print(i, act_segment[player_loc:nxt_stop], "\n")
            all_subsegments.append(act_segment[player_loc:nxt_stop])
        else:
            print(i, act_segment[player_loc:], "\n")
            all_subsegments.append(act_segment[player_loc:])

In [ ]:

#get all player's actions
player_actions = []
for token in story:
    if token.dep_ is "nsubj"and token.text == player_name and token.head.pos is VERB:
        player_actions.append(token.head.text)
#         print(token.text, token.pos_, token.tag_, token.dep_, token.head.text, token.i)

In [ ]:
#Extraction from subsegmentation
#input story
#output subsegments = [action, subsegment, entities, parameters]
# sents = [sent.text for sent in story.sents]
sents = all_subsegments
#take sentences where the player is performing actions
#save the action, the sentence, the entities and their parameters 
subsegments= []
for sent in all_subsegments:
    for token in nlp(sent):
        if token.dep_ is "nsubj" and token.text == player_name and token.head.pos is VERB :
            #get parameters and entity preonditions
            param_ents= []
            r_entities = []
            params = []
            ent_preconds = []
            for entity in Entities:
                e = str(sent).find(entity.lower())
                if e >-1:
                    param_ents.append(entity)
                    r_entities.append(Entities[entity])
                    params.append(label(Entities[entity]))
                    ent_preconds.append(Entities[entity]+ space + label(Entities[entity]))
            s = [token.head, sent, param_ents,r_entities, params, ent_preconds]
            subsegments.append(s)
            
# for i, subsegment in enumerate (subsegments):
#     effects = dob_effects(subsegment)
#     subsegments[i].append(effects) 
for s in subsegments:
    if get_dob_effect(s):
        s.insert(2,get_dob_effect(s) )
    
    
for s in subsegments:
    print(s, "\n", "\n")
len(subsegments)

In [ ]:

#check the verb
#check if the former effect has ?non, if yes, replace with one of the ones here
#also check if the ?xxx is one of the parameters
player_actions = []
pred_entities= []# entities that will be used as predicates
acts_params = []
for i, subseg in enumerate(sents):
    subseg  = nlp(subseg)
    for token in subseg:
        if token.dep_ is "nsubj"and token.text == player_name and token.head.pos is VERB :
            action = token.head
            a = [child for child in action.children if child.dep_ is "prep"]
            if(len(a)):
                parent  = a[0]
                b = [chil for chil in parent.children if chil.dep_ == 'pobj']
                if len(b):
                    acts_params.append([player_name, str(token.head), b[0], i,])
                    if ent_value(b[0]):
                        e = str(ent_value(b[0]))
                        pred_entities.append([get_past(str(token.head)), label(e)])
                        
                        
            c = [child for child in action.children if child.dep_ == 'dobj']
            if(len(c) and ent_value(c[0])):
                acts_params.append([player_name, str(token.head), c[0],i,])
                player_actions.append(str(token.head))
                if ent_value(c):
                    e = str(ent_value(c[0]))
                    pred_entities.append([get_past(str(token.head)), label(e)])
                    
                    
        if token.dep_ == "advcl":
             for child in token.children:
                for entity in Entities:
                    if entity == str(child):
                        if(ent_value(entity)):
                            acts_params.append([player_name, str(token), entity, i])
                            player_actions.append(str(token))
                            if ent_value(entity):
                                e = str(ent_value(entity))
                                pred_entities.append([get_past(token.text), label(e)])
                            break;

In [ ]:

def get_preconditions(precs):
    preconditions = ""
    pars = {}
    for p in precs:
        param = p[p.find(' '):]
        b4_param=p[:p.find(' ')]
        if param in pars:
            pars[param]+=1
            preconditions+= "("+ b4_param+ param +str(pars[param]) + ")"+ space
        else:
            pars[param]= 0
#             print(pars)
            preconditions+= "("+b4_param+ param+ ")"+ space
    return(preconditions)

def get_parameters(params): #adds one to param count
    parameters = "("
    pars = {}
    for p in params:
        if p in pars:
            pars[p]+=1
            parameters+=  p +str(pars[p]) + space
        else:
            pars[p]= 0
            parameters+=  p + space
   
       
    return(parameters+  " ")

def alt_effects(eff, params): #eff is the segment's direct effect e.g read ?inf
    for alt in pred_entities:
        chosen_alt = None
        if eff.find(str(alt[0])) >-1: #find the action in the direct effect
            chosen_alt = alt
            if(chosen_alt != None): #check if the param is part of the params in the same action
                params = str(" ".join(params))
#                 print("this is params", params,chosen_alt[1] )
                if(params.find(chosen_alt[1])>-1):
                    effect = "(" + chosen_alt[0] + space + chosen_alt[1] + ")"
                    return effect
            
        
    return None
states = []
def get_effects(eff, params):
    a = eff.lower().find("?pla")
    b = eff.lower().find("?non")
    c = eff.lower().find("none")
# if it does search the alternative effects . 
    if a>-1 or b>-1 or c>-1:
        effect = alt_effects(eff, params)
        #save to states
        states.append(effect)
        return effect
        
    effect = "(" + str(eff) + ")"
    states.append(effect)
    return effect 
        
def get_operators(subsegments):
    operators = "(:functions" +"\n"+"(total-cost))\n \n"
    for segment in subsegments:
        a = get_effects(segment[2], get_parameters(segment[-2]))
        if(a != None):
            operators+= ("(:action "+lemmat(str(segment[0]))+ "\n  " +
                   ":parameters " + get_parameters(segment[-2])+ ")"+ "\n  "+ 
                   ":precondition (and " + get_preconditions(segment[-1])  +")"+"\n"+ 
                    ":effect (and "+ get_effects(segment[2],segment[-2]) +" (increase (total-cost) " + str(random_cost()) +")))"+"\n"
                    )+"\n"
    
    return operators
print(get_operators(subsegments))

In [ ]:
# predicates
def param_preds(subsegments):
    predicates = []
    for segment in subsegments:
        preds = segment[-1]
        for pred in preds:
            predicates.append("("+pred+")")
    predicates = unduplicate(predicates)
    return predicates
    
#get state preds from al efects
state_preds = unduplicate(states)
state_preds.remove(None)
# print(state_preds)
# param_preds(subsegments)
all_preds = param_preds(subsegments) + state_preds
all_preds

In [ ]:
def get_predicates(preds):
    predicates = ""
    for pred in preds:
        predicates+=pred+"\n"
        
    return( "(define (domain " +storyname+"land)" +"\n"+
            "(:requirements :action-costs)" + "\n"+
            "(:predicates"+ "\n"+
             predicates+")" )

In [ ]:
# => change entities here, puth hyphen for the ones that have space
def init_objects():
    initial_objects = []
    objects = ""
    for entity in Entities:
        if entity.lower() != player_name:
            initial_objects.append(entity.replace(" ","-").lower())
    initial_objects = unduplicate(initial_objects)
    for obj in initial_objects:
        objects+= obj+" "
    return ("(define (problem "+player_name+")"+"\n"+
            "(:domain "+storyname+"land)"+"\n"+
            "(:objects you "+objects+") \n")


def initial_state():
    init_state = ""
    inits =[]
    for entity in Entities:
        if entity.lower() != player_name: 
            inits.append("("+ Entities[entity].replace(" ","-").lower()  + space +entity.replace(" ","-").lower() + ")")
    inits = unduplicate(inits)
    for init in inits:
        init_state+= init+"\n"
    return ("(:init (player you) \n"+"(= (total-cost) 0) \n"+init_state+")\n")

goal_objects =[] 
for entity in Entities:
    goal_objects.append([entity,label(Entities[entity]) ])
print(init_objects())
print(initial_state())
print(goal_objects)

In [ ]:
#get goal object
def gobj():
    random.shuffle(goal_objects)
    return (goal_objects)

goals = []
for i in range(1, 6):
    no = random.randint(0,len(state_preds)-1)
    sp = state_preds[no]
    labl = sp[sp.find(" "):-1]
    act = sp[1:sp.find("?")-1]
    for s in gobj():
        new_s = " ".join(s)
        s_labl = new_s[new_s.find("?")-1:]
#         print(s_labl, labl)
        if (str(s_labl) == str(labl)):
            obj = new_s[:new_s.find(labl)]
            actobj = "("+act+ space+ obj.replace(" ","-")+")"
            goals.append(actobj)
            break;
            
goals = unduplicate(goals)
# print(goals)

def get_goals():
    g = ""
    for goal in goals:
        
        g += goal.lower()+" "
    return("(:goal (and "+g+ "))\n"+
          "(:metric minimize (total-cost))")


all_goals = get_goals()
print(all_goals)

In [ ]:
#domain file
operators = get_operators(subsegments)
predicates = get_predicates(all_preds)

domainfile = predicates + "\n" + operators + ")"
print(domainfile)

In [ ]:
# files
#writing to file
f = open(storyname+"domain.pddl", "w")
f.write(domainfile)
f.close()

In [ ]:
#problem file
problemfile = init_objects() + "\n" + initial_state() + "\n" +  all_goals + ")"

print(problemfile)

In [ ]:
#writing to file
f = open(storyname+"problem.pddl", "w")
f.write(problemfile)
f.close()

In [ ]:
# complete_effects = []
# # save the segment number and the pobjs
# for i, subsegment in enumerate(subsegments):
#     doc = nlp(subsegment[1])

#     pobjs = []
#     act_pobjs = []
#     for token in doc:
#         if token.dep_ is 'pobj' or token.dep_ is 'dobj':
#             pobjs.append([token, token.i])
        
#     for token in doc:
#         if token.dep_ is "nsubj"and token.text == player_name and token.head.pos is VERB :
#                 action = token.head
#                 index = action.i
#                 get_pobj = [[action, pobj[0]] for pobj in pobjs if index<pobj[1]]
                
#                 if(get_pobj):
#                     act_pobjs.append(get_pobj[0])
            
    


# # pobjs = pobjs.sort(key=itemgetter(1))

#     if act_pobjs:
#         complete_effects.append([i, act_pobjs])
# print(complete_effects)